<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 1.0 Transformer Architecture
트랜스포머 아키텍처는 수많은 모던 자연어 처리, 컴퓨터 비전, 자동 음성 인식, 시계열 모델링 및 기타 애플리케이션들의 기반이 되고 있습니다. 이 노트북에서는 이번 수업의 다음 파트에서 BERT 및 GPT 아키텍처의 구현 세부 사항에 대해 논의할 수 있도록 Transformer 아키텍처를 살펴보겠습니다. <br>


**[1.1 Transformer Overview](#1.1-Transformer-Overview)<br>**
**[1.2 The Tokenizer](#1.2-The-Tokenizer)<br>**
**[1.3 Embedding](#1.3-Embedding)<br>**
**[1.4 Positional Encoding](#1.4-Positional-Encoding)<br>**
**[1.5 Transformer Encoder](#1.5-Transformer-Encoder)<br>**
**[1.6 What is Attention?](#1.6-What-is-Attention?)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[1.6.1 Self-Attention](#1.6.1-Self-Attention)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.6.2 Multi-Head Attention](#1.6.2-Multi-Head-Attention)<br>
**[1.7 Transformer Decoder](#1.7-Transformer-Decoder)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[1.7.1 Masked Multi-Head Attention](#1.7.1-Masked-Multi-Head-Attention)<br>
**[1.8 Visualization](#1.8-Visualization)<br>**

---
# 1.1 Transformer Overview

2017년 후반에는 Translation을 포함한 광범위한 시퀀스 투 시퀀스 문제들이 콘볼루션 (Convolution) 또는 Recurrent Neural Network (순환신경망, RNN)를 통해 해결되었습니다 (그림 1 참조). 해당 시기의 모델들이 공통적으로 가지고 있는 점은 둘 다 긴 시퀀스를 처리할 수 있도록 어텐션 (Attention) 메커니즘을 광범위하게 이용했다는 사실입니다. ["Attention is All You Need!"(Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762) 논문에서 소개되었듯이 트랜스포머 (Transformer) 아키텍처는 새로운 접근 방법을 제시하였고, 논문에 따르면 컨볼루션 또는 리커런트 메커니즘의 필요성을 완전히 제거하는 어텐션 메커니즘을 활용하여 유사한 성능을 달성할 수 있음을 증명하였습니다.

트랜스포머는 이후 개발된 많은 모델의 기초가 되었으며 Natural Language Processing (NLP) 작업에도 사용되었지만 현재 컴퓨터 비젼 같은 다른 많은 분야에서 새로운 발전의 물결을 뒷받침하고 있습니다. (논문: ["An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale"](https://arxiv.org/abs/2010.11929)). 왜냐하면 트랜스포머는 BERT와 같은 모델들의 기반이 되기 때문에, 내부 동작 원리를 이해하는 것은 중요합니다. 

<center><figure>
    <img src="images/nlp.png" width="800">
    <figcaption>그림 1. 자연어 처리 분야의 주요 혁신에 대한 타임라인 모음</figcaption>
</figure></center>


트랜스포머는 탁월한 시퀀스 투 시퀀스(seq2seq) 모델입니다. 2017/2018년 이전에 광범위한 시퀀스 모델링 작업을 해결하기 위해 사용된 순환 또는 컨볼루션 뉴럴 네트워크 (RNNs / CNNs)의 대안입니다. 왜냐하면 트랜스포머는 RNN과 CNN에서의 중요한 결점인 즉 연산 비용을 설명할 수 있기 때문입니다. Translation 작업에 초점을 맞춘 원본 논문에서 저자들은 8개의 GPU를 바탕으로 3.5일의 학습만으로 당시 기준 SOTA 모델 성능을 달성했는데, 이 학습시간은 대체 모델들에서 필요한 학습 시간 대비 적은 비율이었습니다. 이러한 연산 자원에 대한 요구사항의 감소는 연구 커뮤니티가 가용한 데이터셋과 모델의 크기를 크게 증가시켜 GPT, BERT, 그리고 그 이상의 아키텍처들의 성공을 이끌 수 있도록 했습니다.

대부분의 seq2seq 모델은 인코더 및 디코더 구성요소로 구성됩니다. 인코더는 입력 시퀀스를 받아 중간 표현, 즉 n차원 벡터로 매핑합니다. 이 추상 벡터는 디코더로 공급되어 출력 시퀀스로 변환됩니다. 그림 2는 시퀀스 투 시퀀스 문제의 예로서 Translation을 사용하는 인코더-디코더 아키텍처를 보여줍니다.


<center><figure>
    <img src="images/enc_dec.png" width="400">
    <figcaption>그림 2. 시퀀스 투 시퀀스 모델</figcaption>
</figure></center>

트랜스포머 아키텍처도 인코더 및 디코더 파트로부터 구축되지만(그림 3 참조), 기존의 RNN 또는 CNN 기반 아키텍처와 달리 트랜스포머는 문장의 모든 단어 간의 관계를 직접 모델링하는 셀프 어텐션 (Self-Attention) 메커니즘에 *완전* 의존합니다.

<center><figure>
    <img src="images/Transformer_architecture.png">
    <figcaption>그림 3. 이미지 크레딧: <a href="https://arxiv.org/abs/1706.03762">Attention is All You Need</a></figcaption>
</figure></center>

리커런트 또는 컨볼루션 시퀀스 투 시퀀스 모델과 비교하여, 트랜스포머는 비교적 간단한 메커니즘으로 구축이 가능합니다. (이 모든 내용은 다음 섹션에서 자세히 설명됨). 
1. 토크나이저 - Tokenizer
2. 임베딩 레이어 - Embedding layers
3. 포지셔널 인코딩 - Positional Encoding
4. 멀티 헤드 어텐션과 마스크 멀티 헤드 어텐션 - Multi-Head Attention and Masked Multi-Head Attention
5. 피드 포워드 레이어 (Feed forward layer) 와 같은 다양한 기본 구성 요소

멀티-헤드 어텐션은 피드 포워드 메커니즘과 함께 여러번 반복할 수 있는 트랜스포머 레이어를 구성하여 아키텍처를 확장하거나 축소할 수 있습니다. 기존 트랜스포머 모델은 인코더에 있는 6개의 동일한 레이어와 모델의 디코더 단계에 있는 6개의 동일한 레이어로 구성되어 있습니다. 

BERT 아키텍처로 이동하기 전에 위에서 언급한 구성요소에 대해 간략히 설명하겠습니다.


---
# 1.2 The Tokenizer 

아키텍처의 기반(그리고 뉴럴 네트워크 자체의 외부)에는 토크나이저가 있습니다. 토크나이저는 원본 텍스트를 토큰(하위 단어 구성 요소)으로 구성된 텍스트의 초기 숫자 표현으로 변환합니다. 이것은 모델이 인간 언어의 복잡성을 우아하게 처리할 수 있도록 하기 때문에 아키텍쳐에서 중요한 부분입니다. 예를 들어, 원래 말뭉치, 외국어/특수 문자에 없는 새로운 단어를 다루거나 결과 텍스트 표현이 간결하게(가능한 한 짧게) 만드는 통합 언어에서 단어를 관리하기 쉬운 구성 요소로 나눕니다. 각 언어마다 매우 많은 수의 토크나이저가 존재하며 이들은 상당히 다릅니다. 대부분의 트랜스포머 기반 아키텍처는 일반적인 시퀀스의 길이를 최소화하도록 설계된 훈련된 토크나이저를 사용합니다. WordPiece(Bert에서 사용) 또는 StentPiece(T5 또는 RoBerta에서 사용)와 같은 토크나이저도 여러 언어, 전문 도메인(예: 의료 코퍼스)에 채택되고 있으며 다른 아키텍처 방식에서도 다양하며 다양한 토큰 번호로 훈련되고 다른 방식으로 또는 같은 방법으로 케이싱된 토큰에서 처리됩니다. 안타깝게도 토크나이저 선택이 모델 동작에 미치는 영향을 설명하는 문헌은 여전히 부족하며 대부분의 경우 원래 모델 구현에 사용된 기본 토크나이저가 사용됩니다. 토크나이저는 모델 아키텍처에서 상대적으로 재학습 및 변경이 쉽습니다.

다음 노트북에서는 BERT 모델의 토크나이저 옵션에 대해 자세히 살펴보고 그 구현에 대해 자세히 알아보겠습니다.


---
# 1.3 Embedding

토큰화된 텍스트는 *임베딩* (Embedding) 메커니즘에 의해 처리됩니다. 임베딩은 원래 데이터를 뉴럴 네트워크에서 사용할 수 있는 수치 표현으로 변환하는 알고리즘에 의해 생성됩니다. 이러한 수치 표현을 흔히 텍스트 임베딩이라고 합니다. 

텍스트 임베딩은 일반적으로 기계 학습 알고리즘이 텍스트의 복잡성을 처리하고 텍스트 표현을 더 간결하게 만드는 데 도움이 되는 추가 속성을 갖도록 설계되었습니다. 예를 들어, [Word2Vec](https://arxiv.org/pdf/1310.4546.pdf), [Globe](https://nlp.stanford.edu/projects/glove/), 또는 [fastText](https://fasttext.cc/) 와 같은 알고리즘/라이브러리는 개별 단어의 의미를 포착하는 임베딩을 생성하여 관련되지 않은 단어가 임베딩 공간에 더 가깝게 나타나도록 합니다.

대조적으로, 트랜스포머 아키텍처는 뉴럴 네트워크 자체가 의미 구조를 학습할 수 있는 충분한 용량을 가지고 있기 때문에 일반적으로 매우 간단하고 학습된 임베딩과 함께 사용됩니다. 따라서 트랜스포머 구현의 임베딩 레이어는 원래 토큰을 타겟 숫자 표현으로 변환하는 매트릭스일 뿐입니다. (매트릭스 사이즈는 다음과 같습니다: $\mathbf{Vocabulary Size}$ X $\mathbf{d_{model}}$). 이 임베딩 매트릭스는 엔드 투 엔드 최적화 프로세스의 일부로 학습됩니다. 원래의 트랜스포머 구현에서 이 매트릭스는 소프트맥스 연산 바로 전의 선형 레이어 뿐만 아니라 입력과 출력 임배딩 사이의 가중치를 공유한다는 점에 주목할 필요가 있습니다. (그림 3의 오른쪽 위 참고)

$\mathbf{Vocabulary Size}$ 와 $\mathbf{d_{model}}$ 는 모두 변경할 수 있는 아키텍처 하이퍼 파라미터입니다. 원래 트랜스포머 모델에서는 $\mathbf{d_{model} = 512}$를 사용하지만 NVIDIA 구현에서는 “Transformer-big” 모델인 $\mathbf{d_{model} = 1024}$ 을 사용합니다.

---
# 1.4 Positional Encoding 

언어 모델은 문장에서 단어의 순차적 특성을 활용할 필요가 있습니다. 트랜스포머 모델에는 순환 또는 컨볼루션 단위가 없으므로 위치 인코딩(Positional Encoding, PE)이 입력 시퀀스의 단어 순서를 설명하는 데 사용됩니다. 위치 인코딩은 임베딩과 동일한 차원, d<sub>model</sub>를 가지므로 둘을 합칠 수 있습니다(그림 3 참조). 이를 통해 모델은 입력 텍스트에서 각 단어의 위치를 이해할 수 있습니다.

기존 트랜스포머 논문의 위치 인코딩은 단순히 사인과 코사인 함수의 조합이며, 이 클래스에서 사용되는 NVIDIA NeMo 라이브러리에서 실제 구현된 내용은 다음과 같습니다. 위치 인코딩을 설명하기 위해 아래 셀을 실행하십시오. 이 계산에 대한 자세한 내용은 옵션으로 진행 가능한 노트북인 [Positional Encoding](more_details/015_PositionalEncoding_Detail.ipynb)에 나와 있습니다. 그리고 데모 예제가 최신 노트북에 포함되어 있습니다.

아래의 위치 인코딩 선택은 상대적으로 임의적입니다. 다행히도 위치 인코딩이 모델 성능에 미치는 영향에 대한 더 많은 연구가 이루어지고 있습니다. 참고 논문 - ["Rethinking Positional Encoding in Language Pre-training"](https://arxiv.org/pdf/2006.15595.pdf)


In [ ]:
# FixedPositionalEncoding implementation in NeMo
# Find more in NeMo/nemo/collections/nlp/modules/common/transformer/transformer_modules.py
import math
import numpy as np
import torch

hidden_size=512        
max_sequence_length=768
pos_enc = torch.zeros(max_sequence_length, hidden_size)
position = torch.arange(0.0, max_sequence_length).unsqueeze(1)
coef = -math.log(10000.0) / hidden_size
div_term = torch.exp(coef * torch.arange(0.0, hidden_size, 2))
pos_enc[:, 0::2] = torch.sin(position * div_term)
pos_enc[:, 1::2] = torch.cos(position * div_term)
pos_enc.div_(math.sqrt(hidden_size))
T=pos_enc.cpu().detach().numpy()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
 
plt.figure(figsize=(50,20))
plt.plot(T)
plt.show()

---
# 1.5 Transformer Encoder
그림 4에서 설명된 인코더 메커니즘 디자인을 확인해 봅시다. 트랜스포머 논문에서는 각 인코더와 디코더가 $\mathbf{N = 6}$의 동일한 레이어로 구성되어 총 12개의 레이어를 구성하고 있습니다. 각 6개의 인코더 레이어는 2개의 서브 레이어를 가지고 있습니다.: 첫번째는 멀티-헤드 셀프-어텐션 메커니즘(multi-head self-attention mechanism)입니다; 주번째는 간단하고 위치기반 완전히 연결된 피드-포워드 네트워크 입니다(simple, position-wise fully connected feed-forward network). 

인코더의 목표는 히든 스테이트 벡터(hidden state vectors)로 소스 문장을 인코드하는 일입니다; 디코더는 스테이트 백터의 마지막 표현을 활용하여 타겟 언어의 문자들을 예측합니다. 단일 인코더 블록(6개 중 1개)이 아래에 표현되어 있습니다:

<center><figure>
    <img src="images/encoder1.png" width="500">
    <figcaption>그림 4. 인코더 블록</figcaption>
</figure></center>

---
# 1.6 What is Attention?

트렌스포머의 백본은 "어텐션(attention)"메커니즘 입니다. 

여기의 의도는 딥러닝에서 어텐션을 사람이 어떻게 시각적인 장면을 바라보는 지에 대한 모방으로 생각합니다. 우리는 우리가 바라볼때 모든 것을들 스캔하지 않습니다. 장면의 컨텍스트에 따라서 중요한 피쳐만을 집중하는 편인데요. 유사하게, 언어에서도 우리는 맥락에 따라 특정한 중요 단어들에 대해서 집중하여 단어들이 어떻게 다른 단어에 적용되는 지 초점을 맞춥니다. (예시로 그림 5를 확인하세요.)

어텐션 메커니즘은 인풋 시퀀스를 바라보고 각 단계마다 문장의 어떤 부분이 중요한 지 결정합니다. 딥러닝의 어텐션은 중요 가중치들의 벡터로 해석될 수도 있습니다. 아래 예제에서 우리는 "ball(공)"이 각 "tennis(테니스)"와 "playing(놀다)" 단어에 강한 어텐션을 가지고 있지만 "tennis(테니스)"와 "dog(개)"는 약하게 연결되어 있음을 확인할 수 있습니다.

<center><figure>
    <img src="images/attention1.png" width="600">
    <figcaption>그림 5. 어텐션 메커니즘 </figcaption>
</figure></center>

## 1.6.1 Self-Attention

트랜스포머 아키텍처에서 어텐션 메커니즘은 3가지 구성 요소를 가지고 있습니다: Q (Query), K (Key), V (Value). 각 3가지 요소들은 연관된 가중치 매트릭스를 가지고 있으며 최적화 프로세스를 통해 학습됩니다. 

셀프-어텐션을 계산하는 첫번째 단계는 각 인코더 인풋 벡터(임베딩)로부터 3가지 벡터를 생성하는 일입니다. 인풋을 적절한 가중치 매트릭스로 곱함으로 각 단어를 위한 Q 벡터, K 벡터, V 벡터가 생성됩니다. 

어텐션 함수는 아래와 같이 정의 될 수 있습니다:

<img src="images/attentionfunction.png" width="400">

Q,K,V 매트릭스가 어떻게 계산되는 지를 확인해 봅시다. 트랜스포머는 Key-Value 쌍(K,V)과 각 d<sub>k</sub> 디멘션으로 구성되어 있는 집합으로 인풋의 인코드된 표현 값을 확인합니다.

1. 각 단어를 위한 Query 벡터 (q<sub>i</sub>), Key 벡터 (k<sub>i</sub>), Value 벡터(v<sub>i</sub>)를 생성합니다. 이 벡터들은 단어의 임베딩을 트레이닝 프로세스 동안 학습된 3가지 메트릭스 값으로 곱함으로 만들어 집니다. 
2. 각 단어를 위한 셀프-어텐션 스코어를 계산합니다. 첫번째 단어의 스코어는 q<sub>1</sub>.k<sub>1</sub>이며 두번째를 위해서 스코어는 q<sub>1</sub>.k<sub>2</sub>입니다.
3.스코어들을 8로 나눕니다.(논문에서는 square root of the dimension of the key vectors $\sqrt{𝑑_k}$이 사용됨)
4. 결과 값을 SoftMax 오퍼레이션으로 보냅니다.
5. 각 Value 벡터를 SoftMax 스코어로 곱합니다: SoftMax x 값.
6. 가중치가 부과된 Value 벡터들을 합을 진행합니다. (첫 번째 단어를 위해) 이 위치에 셀프-어텐션 레이어의 아웃풋을 만듭니다. 

### Self-Attention Matrix Calculation

셀프-어텐션 매트릭스 연산을 세부적으로 살펴봅니다.

아래 그림은 셀프-어텐션의 매트릭스 연산을 나타냅니다. (위의 어텐션 함수와 비교해 봅니다) 첫번째로, 우리는 임베딩들을 매트릭스 <b>X</b>로 압축합니다. 매트릭스 <b>X</b>의 각 열은 인풋 시퀀스의 각 단어의 임베딩 값을 포함하고 있습니다. 예를 들면 </b>X</b> = [<b>x<sub>1</sub></b>, <b>x<sub>2</sub></b>, ..., <b>x<sub>n</sub></b>], 각  <b>x<i><sub>i</sub></i></b>은 <i>i</i> 단어를 나타내는 임베딩 값에 대한 벡터임.


<center><figure>
    <img src="images/attention2.png" width="500">
    <figcaption>그림 6. 셀프-어텐션 매트릭스 연산 공식</figcaption>
</figure></center>

<b>Q</b> 매트릭스는 <b>X</b> 를 $W^Q_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$ 로 곱함으로 계산됩니다.  <b>K</b> 매트릭스는 <b>X</b>를 $W^K_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$를 곱함으로 계산되며 <b>V</b> 매트릭스는 <b>X</b> 를 $W^V_i \in \mathbb{R}^{d_{\text{model}} \times d_v}$ 로 곱해서 계산합니다.


위 공식에서 알 수 있듯이, Value 벡터는 Query와 Key 사이의 multiplication(곱) 값으로 다시 변경됩니다. 이 메커니즘을 트레이닝하면서 Value 벡터의 어떤 부분을 제한하는 방법과 어떻게 다른 부분들을 효과적으로 강화하는 지에 대해 학습하여 모델이 텍스트의 특정 부분에 참여(attend)할 수 있도록 합니다. 

다음 노트북에서는 우리는 이 어텐션 메커니즘을 학습된 BERT 모델을 사용하여 시각화해볼 예정입니다.  


## 1.6.2 Multi-Head Attention

2개의 머리 (Head) 는 하나보다 나을까요? 8개의 머리는 어떨까요? 셀프 어텐션를 정교하게 하는 것을 "멀티-헤드" 어텐션라고 하며, 이를 통해 모델은 다른 위치 또는 하위 공간(sub-spaces)에 초점을 맞출 수 있습니다. 

트랜스포머 아키텍처에서 h = 8 병렬 어텐션 레이어들 또는 헤드들이 있다고 생각해봅시다. 이 것은 8 개 버전의 셀프-어텐션이 동시에 동작하는 것을 의미 합니다. 

<center><figure>
    <img src="images/multiheadattention.png" width="300">
    <figcaption>그림 7. 멀티 헤드 어텐션</figcaption>
</figure></center>

매트릭스 연산에 대한 세부 디테일은 선택 학습 내용인 [Multi-Head Attention](more_details/017_Multi-HeadAttention_Detail.ipynb) 노트북에서 제공됩니다. 

---
# 1.7 Transformer Decoder

디코더는 인코더와 유사한 방식으로 동작하지만 한번에 한 단어를 생성하며 왼쪽에서 오른쪽으로 이동합니다. 디코더는 다른 이전에 생성된 단어들 뿐만 아니라 인코더에 의해 생성된 마지막 표현들까지 바라봅니다. 

<center><figure>
    <img src="images/enc_dec_2.png" width="800">
    <figcaption>그림 8. 디코더 블록</figcaption>
</figure></center>

인코딩 단계를 마친 이후, 디코딩 단계가 시작됩니다. 디코더는 인코더와 매우 유사한 구조를 가지고 있습니다. 각 인코더 레이어의 2개의 서브-레이어 뿐만 아니라 디코더는 세번째 서브-레이어를 추가하여 인코더 스택의 아웃풋에 대한 멀티-헤더 어텐션으로 동작합니다. 인보다와 유사하게 레이어 정규화(Normalization)에 이어 서브-레이어 주변에 레지듀어 커넥션들을(residual connection) 가지고 있습니다. 

## 1.7.1 Masked Multi-Head Attention

그림 3의 디코더에는 마스킹된 멀티-헤드 어텐션(masked multi-head attention)을 확인할 수 있습니다. 디코더의 셀프-어텐션 레이어를 통해 디코더의 각 위치가 해당 위치를 포함한 디코더의 모든 위치에 주의를 기울일 수 있습니다.디코더에서 왼쪽으로 정보가 흐르지 않게 해야 Auto-regressive(자동 퇴행) 특성을 유지할 수 있습니다. 이러한 마스킹은 출력 임베딩이 한 위치만큼 오프셋된다는 사실과 결합되어 위치 <i>i</i>에 대한 예측이 <i>i</i>보다 작은 위치의 알려진 출력에만 의존할 수 있도록 보장합니다. 즉, 마스킹된 멀티-헤드 어텐션은 향후 단어가 어텐션의 일부가 되지 않도록 하기 위해 적용됩니다.


마스킹 기능은 단어와 원본 단어 뒤에 나타나는 단어 사이의 유사성을 제거하는 것입니다("향후에는"). 이러한 정보는 단순히 제거되므로 모델에서 사용할 수 없으며, 이전 단어와의 유사성만 고려됩니다.

---
# 1.8 Visualization
아래 애니메이션은 트랜스포머가 기계 번역에 어떻게 적용되는지 보여줍니다. Transformer는 각 단어에 대해 초기 표현 또는 임베딩을 생성하는 것으로 시작합니다. 이것들은 채워지지 않은 원으로 표현합니다. 그런 다음, 셀프-어텐션을 사용하여 다른 모든 단어에서 정보를 취합하여 전체 컨텍스트가 알려주는 단어당 새로운 표현을 생성합니다. 채워진 공으로 표현됩니다. 그런 다음 이 단계를 모든 단어에 대해 여러 번 동시에 반복하여 연속적으로 새로운 표현을 생성합니다.

<center><figure>
    <img src="https://3.bp.blogspot.com/-aZ3zvPiCoXM/WaiKQO7KRnI/AAAAAAAAB_8/7a1CYjp40nUg4lKpW7covGZJQAySxlg8QCLcBGAs/s1600/transform20fps.gif">
    <figcaption>그림 9.  트랜스포머 단계별 시퀀스를 영어에서 프랑스어로 번역하는 과정 : <a href="https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html">Google Blog</a></figcaption>
</figure></center>

---
<h2 style="color:green;">축하합니다!</h2>

여러분은 다음을 배웠습니다.  
* 트랜스포머가 RNN의 필요성을 제거하여 NMT를 "변환"했습니다.
* 어텐션 메커니즘은 트랜스포머 아키텍처의 핵심입니다.
* 변압기 인코더에는 6개의 (기본값) 누적 인코더 블록이 있습니다.
* 인코더 블록은 셀프 어텐션 및 피드 포워드라는 두 부분으로 구성됩니다.
* 임베딩 알고리즘은 단어의 벡터 표현을 만듭니다.
* 시퀀싱을 위한 RNN이 없으므로 위치 인코딩(Positional Encoding)이 필요합니다.
* 멀티-헤드 어텐션을 통해 모델이 여러 하위 공간(sub-space)에 초점을 맞출 수 있습니다.
* 디코더에는 세 개의 서브 레이어가 있습니다.
* 디코더는 이전에 생성된 단어와 인코더에 의해 생성된 최종 표현에 주의를 기울입니다.
* 디코더는 마스킹된 멀티-헤드 어텐션을 사용하여 향후 단어가 포함되지 않도록 합니다.

휴! 너무 많은 정보들이었네요. 다음 [2.0 BERT](020_BERT.ipynb)노트북으로 가시거나 추가 학습을 위해서 다시 돌아와 주세요. 

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>